# Batch effect evaluation on JUMP Target

In [1]:
%load_ext autoreload
%autoreload 2

## Dataset creation

In [4]:
import logging
import os
import os.path as osp
import shutil
from pathlib import Path
from typing import Any, Callable, Dict, List, Optional

import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import yaml
from lightning import LightningDataModule
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import WandbLogger
from omegaconf import DictConfig, OmegaConf
from torch.utils.data import DataLoader, Dataset
from torchmetrics import MetricCollection
from torchmetrics.classification import MulticlassAccuracy

from src.eval.moa.datamodule import JumpMOADataModule
from src.eval.moa.module import JumpMOAImageGraphModule, JumpMOAImageModule
from src.modules.collate_fn import image_graph_label_collate_function, label_graph_collate_function
from src.modules.compound_transforms import DGLPretrainedFromSmiles
from src.modules.images.timm_pretrained import CNNEncoder
from src.modules.molecules.dgllife_gin import GINPretrainedWithLinearHead
from src.modules.transforms import DefaultJUMPTransform
from src.splitters import StratifiedSplitter
from src.utils.io import download_and_extract_zip, load_image_paths_to_array

In [5]:
for i in range(1, 4):
    if not Path(f"../cpjump{i}/jump/").exists():
        print(f"Mounting cpjump{i}...")
        os.system(f"sshfs bioclust:/projects/cpjump{i}/ ../cpjump{i}")
    else:
        print(f"cpjump{i} already mounted.")

Mounting cpjump1...
Mounting cpjump2...
Mounting cpjump3...


In [6]:
metadata_path = "../cpjump1/jump/metadata"
load_data_path = "../cpjump1/jump/load_data"

In [7]:
os.listdir(metadata_path)

['compound.csv.gz',
 'crispr.csv.gz',
 'microscope_config.csv',
 'microscope_filter.csv',
 'orf.csv.gz',
 'plate.csv.gz',
 'README.md',
 'well.csv.gz',
 'compound.csv',
 'crispr.csv',
 'orf.csv',
 'plate.csv',
 'well.csv',
 'complete_metadata.csv',
 'resolution.csv',
 'JUMP-Target-1_compound_metadata.tsv',
 'JUMP-Target-1_compound_platemap.tsv',
 'JUMP-Target-1_crispr_metadata.tsv',
 'JUMP-Target-1_crispr_platemap.tsv',
 'JUMP-Target-1_orf_metadata.tsv',
 'JUMP-Target-1_orf_platemap.tsv',
 'JUMP-Target-2_compound_metadata.tsv',
 'JUMP-Target-2_compound_platemap.tsv',
 'JUMP-MOA_compound_metadata.tsv',
 'local_metadata.csv']

In [12]:
os.listdir("../cpjump1/hint-clinical-trial-outcome-prediction/data")

[]

In [10]:
os.system("cat ../cpjump1/hint-clinical-trial-outcome-prediction/Contents.txt")

All 457,560 studies in XML format
  Studies grouped by leading part of NCT Id
  Studies published on July 2, 2023


0

In [4]:
load_df = pd.read_parquet(osp.join(load_data_path, "final"))

In [5]:
load_df

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_Site,FileName_OrigAGP,FileName_OrigDNA,FileName_OrigER,FileName_OrigMito,FileName_OrigRNA
0,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A01,1,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...
1,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A01,3,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...
2,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A01,6,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...
3,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A02,1,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...
4,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A02,2,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...
...,...,...,...,...,...,...,...,...,...,...
5121195,source_9,20211103-Run16,GR00004421,Z47,3,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...
5121196,source_9,20211103-Run16,GR00004421,Z47,4,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...
5121197,source_9,20211103-Run16,GR00004421,Z48,1,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...
5121198,source_9,20211103-Run16,GR00004421,Z48,2,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...


In [17]:
meta = pd.read_csv(osp.join(metadata_path, "local_metadata.csv"))

In [22]:
target_local = meta.query("Metadata_PlateType == 'TARGET2'")

In [25]:
target_local

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_PlateType,Metadata_Well,Metadata_JCP2022,Metadata_InChIKey,Metadata_InChI,Metadata_Sites_Per_Well,trt
114388,source_10,2021_08_03_U2OS_48_hr_run12,Dest210726-160150,TARGET2,A01,JCP2022_043547,KBPLFHHGFOOTCA-UHFFFAOYSA-N,"InChI=1S/C8H18O/c1-2-3-4-5-6-7-8-9/h9H,2-8H2,1H3",6,target_trt
114389,source_10,2021_08_03_U2OS_48_hr_run12,Dest210726-160150,TARGET2,A02,JCP2022_050797,LOUPRKONTZGTKE-UHFFFAOYSA-N,InChI=1S/C20H24N2O2/c1-3-13-12-22-9-7-14(13)10...,6,target_trt
114390,source_10,2021_08_03_U2OS_48_hr_run12,Dest210726-160150,TARGET2,A03,JCP2022_050997,LPYXWGMUVRGUOY-UHFFFAOYSA-N,InChI=1S/C6H8O6/c7-1-2(8)5-3(9)4(10)6(11)12-5/...,6,target_trt
114391,source_10,2021_08_03_U2OS_48_hr_run12,Dest210726-160150,TARGET2,A04,JCP2022_108326,YGSDEFSMJLZEOE-UHFFFAOYSA-N,"InChI=1S/C7H6O3/c8-6-4-2-1-3-5(6)7(9)10/h1-4,8...",6,target_trt
114392,source_10,2021_08_03_U2OS_48_hr_run12,Dest210726-160150,TARGET2,A05,JCP2022_033924,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,InChI=1S/C2H6OS/c1-4(2)3/h1-2H3,6,target_neg
...,...,...,...,...,...,...,...,...,...,...
842725,source_9,20211103-Run16,GR00004409,TARGET2,Z44,JCP2022_060040,NMUSYJAQQFHJEW-UHFFFAOYSA-N,InChI=1S/C8H12N4O5/c9-7-10-2-12(8(16)11-7)6-5(...,4,target_trt
842726,source_9,20211103-Run16,GR00004409,TARGET2,Z45,JCP2022_019314,FABUFPQFXZVHFB-UHFFFAOYSA-N,InChI=1S/C27H42N2O5S/c1-15-9-8-10-27(7)22(34-2...,4,target_trt
842727,source_9,20211103-Run16,GR00004409,TARGET2,Z46,JCP2022_018899,DXZRBHUCOHBAHP-UHFFFAOYSA-N,InChI=1S/C15H13Cl2N3O2/c1-3-22-15(21)14-11(8(6...,4,target_trt
842728,source_9,20211103-Run16,GR00004409,TARGET2,Z47,JCP2022_033924,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,InChI=1S/C2H6OS/c1-4(2)3/h1-2H3,4,target_neg


In [7]:
target_meta = pd.read_csv(osp.join(metadata_path, "JUMP-Target-2_compound_metadata.tsv"), sep="\t")

In [10]:
target_meta

,broad_sample,InChIKey,pert_iname,pubchem_cid,target,pert_type,control_type,smiles
0,BRD-K09338665-001-07-1,KBPLFHHGFOOTCA-UHFFFAOYSA-N,1-octanol,957.0,GJB4,trt,NaN,CCCCCCCCO
1,BRD-K48278478-001-01-2,LOUPRKONTZGTKE-AFHBHXEDSA-N,quinine,94175.0,KCNN4,trt,NaN,COc1ccc2nccc([C@@H](O)[C@H]3C[C@@H]4CC[N@]3C[C...
2,BRD-A85242401-001-12-3,KRGQEOSDQHTZMX-IGCYCDGOSA-N,ascorbic-acid,9888239.0,P3H1,trt,NaN,OC[C@H](O)[C@H]1OC(=O)C(=O)C1O
3,BRD-K93632104-001-17-2,YGSDEFSMJLZEOE-UHFFFAOYSA-N,salicylic-acid,118212070.0,AKR1C1,trt,NaN,OC(=O)c1ccccc1O
4,BRD-K57313110-001-06-8,ODHCTXKNWHHXJC-VKHMYHEASA-N,pidolic-acid,7405.0,VEGFA,trt,NaN,OC(=O)[C@@H]1CCC(=O)N1
...,...,...,...,...,...,...,...,...
302,BRD-A69636825-003-04-7,HSUGRBWQSSZJOP-UHFFFAOYSA-N,diltiazem,3076.0,CACNG1,trt,NaN,COc1ccc(cc1)C1Sc2ccccc2N(CCN(C)C)C(=O)C1OC(C)=O
303,BRD-K87782578-001-03-9,KXBDTLQSDKGAEB-UHFFFAOYSA-N,AVL-292,59174488.0,BTK,trt,NaN,COCCOc1ccc(Nc2ncc(F)c(Nc3cccc(NC(=O)C=C)c3)n2)cc1
304,BRD-K98763141-001-30-8,JZFPYUNJRRFVQU-UHFFFAOYSA-N,niflumic-acid,4488.0,UGT1A9,trt,NaN,OC(=O)c1cccnc1Nc1cccc(c1)C(F)(F)F
305,BRD-K19975102-001-02-0,YYDUWLSETXNJJT-MTJSOVHGSA-N,GNF-5837,59397065.0,NTRK1,trt,NaN,Cc1ccc(NC(=O)Nc2cc(ccc2F)C(F)(F)F)cc1Nc1ccc2c(...


In [39]:
merged = pd.merge(
    target_meta.dropna(subset=["target"]),
    target_local,
    left_on=["InChIKey"],
    right_on=["Metadata_InChIKey"],
    how="inner",
)[["target", "smiles", "Metadata_Source", "Metadata_Batch", "Metadata_Plate", "Metadata_Well", "pert_type"]]

In [40]:
merged

,target,smiles,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,pert_type
0,GJB4,CCCCCCCCO,source_10,2021_08_03_U2OS_48_hr_run12,Dest210726-160150,A01,trt
1,GJB4,CCCCCCCCO,source_10,2021_08_09_U2OS_48_hr_run13,Dest210727-153003,A01,trt
2,GJB4,CCCCCCCCO,source_10,2021_08_12_U2OS_48_hr_run15,Dest210803-153958,A01,trt
3,GJB4,CCCCCCCCO,source_10,2021_08_17_U2OS_48_hr_run16,Dest210809-134534,A01,trt
4,GJB4,CCCCCCCCO,source_10,2021_08_20_U2OS_48_hr_run17,Dest210810-173723,A01,trt
...,...,...,...,...,...,...,...
26266,UGT1A9,OC(=O)c1cccnc1Nc1cccc(c1)C(F)(F)F,source_9,20211102-Run15,GR00004395,O30,trt
26267,UGT1A9,OC(=O)c1cccnc1Nc1cccc(c1)C(F)(F)F,source_9,20211103-Run16,GR00004409,AE06,trt
26268,UGT1A9,OC(=O)c1cccnc1Nc1cccc(c1)C(F)(F)F,source_9,20211103-Run16,GR00004409,AE30,trt
26269,UGT1A9,OC(=O)c1cccnc1Nc1cccc(c1)C(F)(F)F,source_9,20211103-Run16,GR00004409,O06,trt


In [41]:
target_load_df = pd.merge(
    load_df,
    merged,
    on=["Metadata_Source", "Metadata_Batch", "Metadata_Plate", "Metadata_Well"],
    how="inner",
)

In [49]:
target_load_df

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_Site,FileName_OrigAGP,FileName_OrigDNA,FileName_OrigER,FileName_OrigMito,FileName_OrigRNA,target,smiles,pert_type
0,source_10,2021_08_03_U2OS_48_hr_run12,Dest210726-160150,A01,1,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,GJB4,CCCCCCCCO,trt
1,source_10,2021_08_03_U2OS_48_hr_run12,Dest210726-160150,A01,2,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,GJB4,CCCCCCCCO,trt
2,source_10,2021_08_03_U2OS_48_hr_run12,Dest210726-160150,A01,3,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,GJB4,CCCCCCCCO,trt
3,source_10,2021_08_03_U2OS_48_hr_run12,Dest210726-160150,A01,4,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,GJB4,CCCCCCCCO,trt
4,source_10,2021_08_03_U2OS_48_hr_run12,Dest210726-160150,A01,5,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,GJB4,CCCCCCCCO,trt
...,...,...,...,...,...,...,...,...,...,...,...,...,...
144081,source_9,20211103-Run16,GR00004409,Z40,4,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,ATM,O=c1cc(oc(c1)-c1cccc2Sc3ccccc3Sc12)N1CCOCC1,trt
144082,source_9,20211103-Run16,GR00004409,Z42,1,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,PTK2B,Cc1oncc1C(=O)Nc1ccc(cc1)C(F)(F)F,trt
144083,source_9,20211103-Run16,GR00004409,Z42,2,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,PTK2B,Cc1oncc1C(=O)Nc1ccc(cc1)C(F)(F)F,trt
144084,source_9,20211103-Run16,GR00004409,Z42,3,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,PTK2B,Cc1oncc1C(=O)Nc1ccc(cc1)C(F)(F)F,trt


## Datamodule

## Module

## Evaluator